In [1]:
import numpy as np
import pandas as pd
from nltk import FreqDist
from nltk.tokenize import \
    regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_confusion_matrix
import re
from collections import OrderedDict, defaultdict, Counter
import itertools
import string
import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_colwidth = 150
seed = 7

In [2]:
# Loading dataset

In [3]:
df = pd.read_csv('./data/disaster_tweets/train.csv')
df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires,1
6,10,NaN,NaN,"#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas",1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in the woods...,1
8,14,NaN,NaN,There's an emergency evacuation happening now in the building across the street,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our area...,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [5]:
df.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [6]:
# Keyword and location columns

In [7]:
# Location column doesn't seem to have usable information. In some cases it's nonsense.

In [8]:
print('Sample of some of the location values')
df['location'].unique()[7:17].tolist()

Sample of some of the location values


['World Wide!!',
 'Paranaque City',
 'Live On Webcam',
 'milky way',
 'GREENSBORO,NORTH CAROLINA',
 'England.',
 'Sheffield Township, Ohio',
 'India',
 'Barbados',
 'Anaheim']

In [9]:
Counter(df['keyword']).most_common(20)

[(nan, 61),
 ('fatalities', 45),
 ('armageddon', 42),
 ('deluge', 42),
 ('body%20bags', 41),
 ('damage', 41),
 ('harm', 41),
 ('sinking', 41),
 ('collided', 40),
 ('evacuate', 40),
 ('fear', 40),
 ('outbreak', 40),
 ('siren', 40),
 ('twister', 40),
 ('windstorm', 40),
 ('collision', 39),
 ('derailment', 39),
 ('earthquake', 39),
 ('explosion', 39),
 ('famine', 39)]

In [10]:
# Features and Target

In [11]:
tweets = df[['text']]
target = df['target']

In [12]:
print('Distribution of Target')
print
print(target.value_counts())
print()
print(target.value_counts(normalize=True))

Distribution of Target
0    4342
1    3271
Name: target, dtype: int64

0    0.57034
1    0.42966
Name: target, dtype: float64


In [13]:
# Train Test Split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    tweets, target, test_size=.25, random_state=seed
)

In [15]:
# Cleaning and tokenizing the tweet text

In [16]:
# Saving a copy of untouched tweets
X_train_tweets_unprocessed = X_train.copy()['text'] 

In [17]:
X_train['text'] = X_train['text'].apply(lambda x: x.lower())

<ipython-input-17-cbb6d635a30d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['text'] = X_train['text'].apply(lambda x: x.lower())


In [18]:
X_train.head(10)

,text
1489,@masochisticmage + catastrophe! it caused people to get reckless and the bottom line is that at least three of your friends will have +
5973,#nochilllukehammings\nim screaming
7589,omg earthquake
3788,it's never a good sign when you pull up to work &amp; there's five ambulances &amp; a fire truck in the bay. wompppp at least it's friday
825,my mic and controllers aren't working one second
4569,my baby girls car wreak this afternoon thank god no serious injuries and she was wearing her seatbelt!!!... http://t.co/njqv45nds2
6371,look at the previous battles. citizens were committing suicide so to not be under american control. the bomb was the only way. @nbcnews
4648,@mistresspip i'm amazed you have not been inundated mistress.
1812,maj muzzamil pilot offr of mi-17 crashed near mansehra today. http://t.co/kl4r1ccwct
492,christian attacked by muslims at the temple mount after waving israeli flag via pamela geller - ... http://t.co/f5miuhqaby


In [19]:
# Remove URLs

In [20]:
# Preserve if tweet has a URL. Will use this later.

def binary_url(text):
    """
    Returns 1 if a string contains a URL, else returns 0.
    """
    search = re.search(pattern=r'http\S+', string=text)
    return int(bool(search))

has_url_Series_train = X_train['text'].apply(binary_url).rename('has_url')

In [21]:
X_train['text'] = X_train['text'].apply(
    lambda x: re.sub(pattern=r'http\S+', repl='', string=x)
    )

<ipython-input-21-d605797eee16>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['text'] = X_train['text'].apply(


In [22]:
# Tokenizing

In [23]:
example_tweet = X_train['text'].loc[2372]
example_tweet

"such activities of govt can't derail us from our aim &amp; we still remain peaceful and unite for #freesikhpoliticalprisnors &amp; @bapusuratsingh"

In [24]:
X_train['text'].sample(5)

6073                  rt twit_san_diego 'possible sinkhole disrupts trolley service: a depression in a portion of asphalt in downtown saû_ 
2078                                            @kg4vaal lmaov.v hard the 'ny' is the the new trend babalmao...welcome to nyozi kwaaaaa#dead
3513                      #eyewitness media is actively embraced by #uk audiences. read the report by @emhub on the impact of #ugc in news: 
2520                                                                fear and panic in the air i want to be free from desolation and despair!
2405    @jozerphine literally just look that up! yeah derailed at smithsonian so everythign is shut down from federal center sw to mcpherson
Name: text, dtype: object

In [25]:
token_pattern = r"[a-zA-Z]+'?[a-zA-Z]+"
# Pattern: Any alphanumeric word with at least two characters, including up to one apostrophy

tokenizer = RegexpTokenizer(token_pattern)

In [26]:
tokenizer.tokenize(example_tweet)

['such',
 'activities',
 'of',
 'govt',
 "can't",
 'derail',
 'us',
 'from',
 'our',
 'aim',
 'amp',
 'we',
 'still',
 'remain',
 'peaceful',
 'and',
 'unite',
 'for',
 'freesikhpoliticalprisnors',
 'amp',
 'bapusuratsingh']

In [27]:
# seems to work
# But "amp" is an artifact of "&amp;" which is the HTML entity for "&"
# Replace it with "and" in original text
# Then tokenize

In [28]:
X_train['text'] = X_train['text'].apply(lambda x: re.sub("&amp;", "and", x))

<ipython-input-28-f7ec564c15b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['text'] = X_train['text'].apply(lambda x: re.sub("&amp;", "and", x))


In [29]:
X_train['text_tokenized'] = X_train['text'].apply(tokenizer.tokenize)

<ipython-input-29-1675f3322f1d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['text_tokenized'] = X_train['text'].apply(tokenizer.tokenize)


In [30]:
# Remove stopwords
# Get comprehensive list
# Combine stopwords from two libraries: NLTK and SpaCy

In [31]:
sw_spacy = list(nlp.Defaults.stop_words)
sw_nltk = stopwords.words('english')
stopword_list = list(set(sw_spacy + sw_nltk))

In [32]:
X_train['text_tokenized_no_sw'] = \
    X_train['text_tokenized'].apply(
        lambda x: [w for w in x if not w in stopword_list]
    )

<ipython-input-32-968b6c5a6f45>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['text_tokenized_no_sw'] = \


In [33]:
# Check most common words

In [34]:
FreqDist(X_train['text_tokenized_no_sw'].explode()).most_common(20)

[('like', 264),
 ('fire', 200),
 ("i'm", 184),
 ('new', 176),
 ('news', 164),
 ('people', 151),
 ('video', 119),
 ('disaster', 115),
 ('police', 110),
 ('emergency', 108),
 ('time', 101),
 ('body', 95),
 ('suicide', 90),
 ('california', 90),
 ('storm', 90),
 ('burning', 89),
 ('rt', 88),
 ('crash', 87),
 ('world', 84),
 ('man', 83)]

In [35]:
# "i'm" should also be considered a stop word

In [36]:
stopword_list.extend(["i'm"])

In [37]:
X_train['text_tokenized_no_sw'] = \
    X_train['text_tokenized'].apply(
        lambda x: [w for w in x if not w in stopword_list]
    )

<ipython-input-37-968b6c5a6f45>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['text_tokenized_no_sw'] = \


In [38]:
top_20_no_sw = FreqDist(X_train['text_tokenized_no_sw'].explode()).most_common(20)
top_20_no_sw

[('like', 264),
 ('fire', 200),
 ('new', 176),
 ('news', 164),
 ('people', 151),
 ('video', 119),
 ('disaster', 115),
 ('police', 110),
 ('emergency', 108),
 ('time', 101),
 ('body', 95),
 ('suicide', 90),
 ('california', 90),
 ('storm', 90),
 ('burning', 89),
 ('rt', 88),
 ('crash', 87),
 ('world', 84),
 ('man', 83),
 ('bomb', 82)]

In [39]:
X_train.head()

,text,text_tokenized,text_tokenized_no_sw
1489,@masochisticmage + catastrophe! it caused people to get reckless and the bottom line is that at least three of your friends will have +,"[masochisticmage, catastrophe, it, caused, people, to, get, reckless, and, the, bottom, line, is, that, at, least, three, of, your, friends, will,...","[masochisticmage, catastrophe, caused, people, reckless, line, friends]"
5973,#nochilllukehammings\nim screaming,"[nochilllukehammings, im, screaming]","[nochilllukehammings, im, screaming]"
7589,omg earthquake,"[omg, earthquake]","[omg, earthquake]"
3788,it's never a good sign when you pull up to work and there's five ambulances and a fire truck in the bay. wompppp at least it's friday,"[it's, never, good, sign, when, you, pull, up, to, work, and, there's, five, ambulances, and, fire, truck, in, the, bay, wompppp, at, least, it's,...","[good, sign, pull, work, there's, ambulances, fire, truck, bay, wompppp, friday]"
825,my mic and controllers aren't working one second,"[my, mic, and, controllers, aren't, working, one, second]","[mic, controllers, working, second]"


In [40]:
# Linguistic Feature engineering 

In [41]:
# Using SpaCy to lemmatize our text
# Using SpaCy to count NER tags and POS tags
# NER tags I'm interested in:
## GPE, LOC, NORP, FAC, EVENT, ORG

In [42]:
# Lemmatize

def spacy_lemmatize(tokens):
    doc = nlp(' '.join(tokens))
    return [t.lemma_ for t in doc]

X_train['lemmas'] = X_train['text_tokenized'].apply(spacy_lemmatize)
X_train['lemmas_no_sw'] = X_train['lemmas'].apply(
    lambda x: [l for l in x if l not in stopword_list]
    )

<ipython-input-42-54e8c2b91794>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['lemmas'] = X_train['text_tokenized'].apply(spacy_lemmatize)
<ipython-input-42-54e8c2b91794>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['lemmas_no_sw'] = X_train['lemmas'].apply(


In [43]:
X_train.head()

,text,text_tokenized,text_tokenized_no_sw,lemmas,lemmas_no_sw
1489,@masochisticmage + catastrophe! it caused people to get reckless and the bottom line is that at least three of your friends will have +,"[masochisticmage, catastrophe, it, caused, people, to, get, reckless, and, the, bottom, line, is, that, at, least, three, of, your, friends, will,...","[masochisticmage, catastrophe, caused, people, reckless, line, friends]","[masochisticmage, catastrophe, it, cause, people, to, get, reckless, and, the, bottom, line, be, that, at, least, three, of, your, friend, will, h...","[masochisticmage, catastrophe, cause, people, reckless, line, friend]"
5973,#nochilllukehammings\nim screaming,"[nochilllukehammings, im, screaming]","[nochilllukehammings, im, screaming]","[nochilllukehamming, I, m, scream]","[nochilllukehamming, I, scream]"
7589,omg earthquake,"[omg, earthquake]","[omg, earthquake]","[omg, earthquake]","[omg, earthquake]"
3788,it's never a good sign when you pull up to work and there's five ambulances and a fire truck in the bay. wompppp at least it's friday,"[it's, never, good, sign, when, you, pull, up, to, work, and, there's, five, ambulances, and, fire, truck, in, the, bay, wompppp, at, least, it's,...","[good, sign, pull, work, there's, ambulances, fire, truck, bay, wompppp, friday]","[it, be, never, good, sign, when, you, pull, up, to, work, and, there, be, five, ambulance, and, fire, truck, in, the, bay, wompppp, at, least, it...","[good, sign, pull, work, ambulance, fire, truck, bay, wompppp, friday]"
825,my mic and controllers aren't working one second,"[my, mic, and, controllers, aren't, working, one, second]","[mic, controllers, working, second]","[my, mic, and, controller, be, not, work, one, second]","[mic, controller, work, second]"


In [44]:
# Spacy POS

def helper_untokenize(token_list):
    """
    Helper function.
    Takes in a list of tokens and combines them as a string.
    Instead of having to repeatedly type `lambda x: ' '.join()`
    """
    return ' '.join(token_list)

def helper_spacy_pos(text):
    """
    Helper function.
    Takes in a string and returns a list of part-of-speech tokens.
    """
    doc = nlp(text)
    pos_tokens = [t.pos_ for t in doc]
    return pos_tokens

def make_pos_tokens_df(text_column):
    """
    Takes in a Series of texts.
    Uses SpaCy to turn the text into part-of-speech tokens.
    Uses sklearn's CountVectorizer to count POS tags for each text. 
    """
    cvec = CountVectorizer(tokenizer=helper_spacy_pos)
    pos_vectorized = cvec.fit_transform(text_column)
    pos_vectorized_df = pd.DataFrame(
        pos_vectorized.toarray(),
        columns=cvec.get_feature_names(),
        index=text_column.index
        )
    return pos_vectorized_df

In [45]:
pos_vectorized_df_train = make_pos_tokens_df(X_train['text_tokenized'].apply(helper_untokenize))
pos_vectorized_df_train.head()

,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,VERB,X
1489,2,1,2,2,1,1,0,5,1,1,2,0,0,1,3,0
5973,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0
7589,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3788,2,4,1,2,2,1,0,6,1,0,4,2,0,1,2,0
825,0,0,0,1,1,0,0,3,1,1,1,0,0,0,1,0


In [46]:
# Spacy NER

def helper_spacy_ner(
    text, 
    ner_tags=['GPE', 'LOC', 'NORP', 'EVENT', 'ORG', 'FAC']
    ):
    """
    Helper function.
    Takes in a string and returns a list of named-entity recognition tags.
    Also takes in a specific list of NER tags to look for.
    To look for all NER tags supported by SpaCy, set `ner_tags=None`.
    """
    doc = nlp(text)
    ents = doc.ents
    if ner_tags:
        tags = [ent.label_ for ent in doc.ents if ent.label_ in ner_tags]
        return tags
    else:
        tags = [ent.label_ for ent in doc.ents]
        return tags

def make_ner_tokens_df(text_column):
    """
    Takes in a Series of texts.
    Uses SpaCy to turn the text a list of named entity recognition tags.
    Uses sklearn's CountVectorizer to count NER tags for each text.
    """
    cvec = CountVectorizer(tokenizer=helper_spacy_ner)
    ner_vectorized = cvec.fit_transform(text_column)
    ner_vectorized_df = pd.DataFrame(
        ner_vectorized.toarray(),
        columns=cvec.get_feature_names(),
        index=text_column.index
    )
    return ner_vectorized_df

In [47]:
ner_vectorized_df_train = make_ner_tokens_df(X_train['text_tokenized'].apply(helper_untokenize))
ner_vectorized_df_train.head()

,EVENT,FAC,GPE,LOC,NORP,ORG
1489,0,0,0,0,0,0
5973,0,0,0,0,0,0
7589,0,0,0,0,0,0
3788,0,0,0,1,0,0
825,0,0,0,0,0,0


In [ ]:
# Meta Feature engineering

# include or exclude stop words?


In [ ]:
# has_url_Series already exists
# Start a DF with it

engineered_features_df_train = pd.DataFrame(has_url_Series_train)

# Character count. Original tweet. No URL.
engineered_features_df_train['character_count'] = \
    X_train['text'].apply(len)

# Token count. Including stop words.
engineered_features_df_train['token_count'] = \
    X_train['text_tokenized'].apply(len)

# Number of unique tokens. Including stop words.
engineered_features_df_train['unique_tokens'] = \
    X_train['text_tokenized'].apply(lambda x: len(set(x)))

# Number of lemmas
# Unique lemmas
# Average lemma length

# Average token length. Including stop words.
engineered_features_df_train['mean_token_length'] = \
    X_train['text_tokenized'].apply(lambda x: np.mean([len(t) for t in x]))